In [ ]:
import numpy as np
import pandas as pd
import healpy as hp
from statistics import mean, stdev
from joblib import Parallel, delayed
from tqdm import tqdm
import os

# Create directory for intermediate output
file_inter = "inter_data1"
os.makedirs(file_inter, exist_ok=True)

# Parameters
nside = 8  # Healpix resolution
nf = 100   # Number of bootstrap samples
nbin = 30  # Number of radial bins
use_cores = 15
# Compute Rényi entropy for a given dataset
def compute_renyi_entropy(input_file, output_file, rmin, rmax, Nside=nside, nbin=30):
    Npix = hp.nside2npix(Nside)
    
    # Load input data
    df = pd.read_csv(input_file, sep="\t", header=None)
    df.columns = ['r', 'th', 'ph']
    r1 = df['r'].to_numpy()
    th1 = df['th'].to_numpy()
    ph1 = df['ph'].to_numpy()

    dr = (rmax - rmin) / nbin
    m = np.zeros((nbin, Npix), dtype=float)
    Neff = np.zeros(nbin, dtype=int)

    # Fill Healpix map
    for rr, tt, pp in zip(r1, th1, ph1):
        for j in range(nbin):
            if rmin <= rr <= (rmin + (j+1) * dr):
                px = hp.ang2pix(Nside, tt, pp)
                m[j][px] += 1
                Neff[j] += 1

    # Normalize to probability
    p = np.zeros((nbin, Npix), dtype=float)
    for i in range(nbin):
        if Neff[i] > 0:
            p[i] = m[i] / Neff[i]

    # Compute Rényi entropies for q=1 to 5
    H = np.zeros((nbin, 5), dtype=float)
    for k in range(5):
        q = k + 1
        for i in range(nbin):
            if q == 1:
                H[i][k] = -np.sum(p[i][p[i] > 0] * np.log10(p[i][p[i] > 0]))
            else:
                h_q = np.sum(p[i] ** q)
                if h_q > 0:
                    H[i][k] = np.log10(h_q) / (1 - q)

    # Radius bins
    R = rmin + (np.arange(1, nbin + 1) * dr)
    
    # Save output
    df_out = pd.concat([pd.DataFrame(R), pd.DataFrame(H)], axis=1)
    df_out.to_csv(output_file, sep='\t', header=False, index=False)

# Helper function for parallel processing
def process_sample(n, l, df_base, R_min, R_max, file_inter):
    fd = df_base.sample(frac=0.8, replace=True, random_state=l)
    f_samp = f"{file_inter}/sample{n+1}_{l+1}.dat"
    f_out2 = f"{file_inter}/anis_s{n+1}_{l+1}.dat"
    fd.to_csv(f_samp, sep="\t", header=None, index=False)
    compute_renyi_entropy(f_samp, f_out2, R_min[n], R_max[n])

# Main loop for each input dataset (3 samples)
R_max = np.zeros(3)
R_min = np.zeros(3)

for n in range(3):
    # Input file
    f_in = f'../../../data/mask1/masked_sample_{n+1}.dat'
    df = pd.read_csv(f_in, sep="\t", header=None)
    df.columns = ['r', 'th', 'ph']
    
    # Compute global r min/max
    R_max[n] = df['r'].max()
    R_min[n] = df['r'].min()

    # Full-sample entropy
    f_out1 = f"{file_inter}/renyi_anis_s{n+1}.dat"
    compute_renyi_entropy(f_in, f_out1, R_min[n], R_max[n])

    # Bootstrap samples (parallelized)
    Parallel(n_jobs=use_cores, backend='loky')(
        delayed(process_sample)(n, l, df, R_min, R_max, file_inter)
        for l in tqdm(range(nf), desc=f"Processing Sample {n+1}"))
#######################################################################################################################

nbin= 30
for n in range(3):
    file0 = 'inter_data1/renyi_anis_s'+ str(n+1)+'.dat'
    raw_data = np.loadtxt(file0)

    s1, s2, s3, s4, s5 = raw_data[:, 1], raw_data[:, 2], raw_data[:, 3], raw_data[:, 4], raw_data[:, 5]

    a1 = np.zeros((nf, nbin))
    a2 = np.zeros((nf, nbin))
    a3 = np.zeros((nf, nbin))
    a4 = np.zeros((nf, nbin))
    a5 = np.zeros((nf, nbin))
    RR = np.zeros((nf, nbin))

    for f in range(nf):
        fname = 'inter_data1/anis_s'+ str(n+1) + '_' + str (f+1) + '.dat'
        data = np.loadtxt(fname)

        RR[f] = data[:, 0]
        a1[f] = data[:, 1]
        a2[f] = data[:, 2]
        a3[f] = data[:, 3]
        a4[f] = data[:, 4]
        a5[f] = data[:, 5]

    r_ = RR[1]  # common radius

    # standard deviations at each bin
    std_a1 = np.array([stdev(a1[:, i]) for i in range(nbin)])
    std_a2 = np.array([stdev(a2[:, i]) for i in range(nbin)])
    std_a3 = np.array([stdev(a3[:, i]) for i in range(nbin)])
    std_a4 = np.array([stdev(a4[:, i]) for i in range(nbin)])
    std_a5 = np.array([stdev(a5[:, i]) for i in range(nbin)])

    df_out = pd.DataFrame({
        'r': r_,
        'a1': s1,
        'a2': s2,
        'a3': s3,
        'a4': s4,
        'a5': s5,
        'std_a1': std_a1,
        'std_a2': std_a2,
        'std_a3': std_a3,
        'std_a4': std_a4,
        'std_a5': std_a5
    })

    f_name1 = 'inter_data1/sample'+str(n+1)+'_entr_err.csv' 
    df_out.to_csv(f_name1,index=False)

#############################################################################################################
for n in range(3):
    f_out1 = 'inter_data1/renyi_anis_s'+ str(n+1)+'.dat'
    os.remove(f_out1)
    
    for l in range(nf):
        file_name = 'inter_data1/anis_s'+ str(n+1) + '_' + str (l+1) + '.dat'
        os.remove(file_name)
        f_samp = 'inter_data1/sample'+ str(n+1) + '_' + str(l+1) + '.dat' 
        os.remove(f_samp)
